# LangGraph Fundamentals: StateGraph and Conditional Edges

This notebook introduces the core concepts of LangGraph:
- **StateGraph**: The foundation of LangGraph applications
- **State Management**: How data flows through your graph
- **Nodes**: Individual processing units
- **Conditional Edges**: Dynamic routing based on state
- **Graph Compilation**: Converting your graph definition into an executable application

## What is LangGraph?

LangGraph is a library for building stateful, multi-actor applications with LLMs. It extends LangChain's expression language with the ability to coordinate multiple chains (or actors) across multiple steps of computation in a cyclic manner.

### Key Concepts:
- **State**: Shared data structure that persists across nodes
- **Nodes**: Functions that process and modify the state
- **Edges**: Connections between nodes that determine execution flow
- **Conditional Edges**: Dynamic routing based on state values

## 1. Essential Imports

Let's start by importing the necessary components from LangGraph:

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from IPython.display import Image, display

## 2. Defining the State

The **State** is the central data structure that flows through your graph. It's shared across all nodes and persists throughout the execution.

### Why TypedDict?
- **Type Safety**: Ensures your state has the expected structure
- **IDE Support**: Better autocomplete and error detection
- **Documentation**: Makes it clear what data your graph expects

### Alternative Approaches:
- **Regular Dict**: Less type safety but more flexible
- **Pydantic Models**: More validation but heavier
- **Custom Classes**: Maximum control but more complex

In [ ]:
class MyState(TypedDict):
    """State definition for our counter example.
    
    This state will track a single integer that gets incremented
    until it reaches a certain threshold.
    """
    count: int

## 3. Creating Nodes

**Nodes** are functions that:
1. Receive the current state as input
2. Process or modify the state
3. Return the updated state (or partial state)

### Node Function Requirements:
- Must accept state as first parameter
- Must return a dictionary that updates the state
- Should be pure functions when possible (no side effects)

### Best Practices:
- Keep nodes focused on a single responsibility
- Use descriptive function names
- Add type hints for better code clarity

In [ ]:
def increaseCount(state: MyState) -> MyState:
    """Increment the count by 1.
    
    This node demonstrates basic state modification.
    It takes the current count and increases it by 1.
    
    Args:
        state: Current state containing the count
        
    Returns:
        Updated state with incremented count
    """
    print(f"Current count: {state['count']} -> {state['count'] + 1}")
    
    # Return the updated state
    # Note: We can return just the fields we want to update
    return {"count": state["count"] + 1}

## 4. Conditional Logic Functions

**Conditional functions** determine the next step in your graph based on the current state. They're used with `add_conditional_edges()` to create dynamic routing.

### Key Points:
- Must return a string that matches an edge mapping
- Should be deterministic (same input = same output)
- Can return special values like `END` to terminate the graph

### When to Use Conditional Edges:
- **Loops**: When you need to repeat operations
- **Branching**: When different conditions require different processing
- **Termination**: When you need to stop based on certain criteria

In [ ]:
def condition(state: MyState) -> str:
    """Determine whether to continue incrementing or stop.
    
    This function implements the termination logic for our counter.
    It demonstrates how to create loops and conditional termination.
    
    Args:
        state: Current state containing the count
        
    Returns:
        "continue" if count < 3, "stop" otherwise
    """
    if state["count"] < 3:
        print(f"Count is {state['count']}, continuing...")
        return "continue"
    else:
        print(f"Count reached {state['count']}, stopping.")
        return "stop"

## 5. Building the Graph

Now we'll construct our StateGraph by:
1. Creating a StateGraph instance with our state type
2. Adding nodes (processing functions)
3. Setting the entry point
4. Adding conditional edges for dynamic routing
5. Compiling the graph into an executable application

### Graph Components:
- **StateGraph(MyState)**: Creates a graph that manages MyState
- **add_node()**: Registers a function as a processing node
- **set_entry_point()**: Defines where execution begins
- **add_conditional_edges()**: Creates dynamic routing based on conditions
- **compile()**: Converts the graph definition into an executable app

In [ ]:
def get_graph():
    """Build and return a compiled StateGraph.
    
    This function demonstrates the complete graph construction process:
    1. Initialize the graph with our state type
    2. Add our processing node
    3. Set the starting point
    4. Add conditional routing logic
    5. Compile into an executable application
    
    Returns:
        Compiled graph ready for execution
    """
    # Step 1: Create StateGraph with our state type
    graph = StateGraph(MyState)
    
    # Step 2: Add our increment node
    graph.add_node("increment", increaseCount)
    
    # Step 3: Set entry point (where execution starts)
    graph.set_entry_point("increment")
    
    # Step 4: Add conditional edges for dynamic routing
    graph.add_conditional_edges(
        "increment",  # From this node
        condition,    # Use this function to decide
        {
            "continue": "increment",  # Loop back to increment
            "stop": END               # Terminate the graph
        }
    )
    
    # Step 5: Compile the graph
    app = graph.compile()
    return app

## 6. Graph Visualization

LangGraph provides built-in visualization capabilities to help you understand your graph structure.

### Visualization Benefits:
- **Debug Complex Flows**: See how data moves through your graph
- **Documentation**: Visual representation for team members
- **Validation**: Ensure your graph structure matches your intentions

### Alternative Visualization Methods:
- `draw_ascii()`: Text-based visualization for terminals
- `draw_mermaid()`: Mermaid diagram format
- `draw_mermaid_png()`: PNG image (requires additional dependencies)

In [ ]:
# Create our graph
graph = get_graph()

# Display the graph structure visually
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Visualization error: {e}")
    print("\nGraph structure (text format):")
    print("START -> increment -> [condition] -> increment (if continue) or END (if stop)")

## 7. Graph Execution

Now let's execute our graph and see it in action!

### Execution Methods:
- **invoke()**: Run the graph once with initial state
- **stream()**: Get intermediate results as the graph executes
- **ainvoke()**: Async version of invoke
- **astream()**: Async version of stream

### What Happens During Execution:
1. Graph starts with our initial state `{"count": 0}`
2. Executes the "increment" node (count becomes 1)
3. Runs the condition function (returns "continue")
4. Routes back to "increment" node (count becomes 2)
5. Runs condition again (returns "continue")
6. Routes back to "increment" node (count becomes 3)
7. Runs condition again (returns "stop")
8. Routes to END, terminating execution

In [ ]:
# Define our initial state
initial_state = {
    "count": 0
}

print("Starting graph execution...")
print(f"Initial state: {initial_state}")
print("\n--- Execution Log ---")

# Execute the graph
final_result = graph.invoke(initial_state)

print("\n--- Final Result ---")
print(f"Final state: {final_result}")

## 8. Streaming Execution (Optional)

For more detailed insight into the execution process, we can use streaming to see intermediate states:

In [ ]:
print("\n=== Streaming Execution ===")
print("This shows each step of the graph execution:\n")

# Reset initial state
initial_state = {"count": 0}

# Stream the execution to see intermediate steps
for i, step in enumerate(graph.stream(initial_state)):
    print(f"Step {i + 1}: {step}")

## Key Takeaways

### What We Learned:
1. **StateGraph Basics**: How to create and structure a LangGraph application
2. **State Management**: Using TypedDict for type-safe state definitions
3. **Node Functions**: Creating processing units that modify state
4. **Conditional Routing**: Using conditional edges for dynamic flow control
5. **Graph Compilation**: Converting definitions into executable applications
6. **Execution Methods**: Different ways to run your graph

### When to Use This Pattern:
- **Iterative Processing**: When you need to repeat operations until a condition is met
- **State Machines**: When your application has distinct states and transitions
- **Multi-Step Workflows**: When you need to coordinate multiple processing steps

### Next Steps:
- **Notebook 2**: Learn about structured output and tool integration
- **Notebook 3**: Build chatbots with LangGraph
- **Notebook 4**: Add memory and persistence to your applications

### Common Patterns:
- **Validation Loops**: Keep processing until data meets criteria
- **Retry Logic**: Attempt operations with fallback strategies
- **Progressive Enhancement**: Gradually improve results through iterations